Several reference:
- cross section, https://sites.ualberta.ca/~gingrich/courses/phys512/node103.html
- units, https://en.wikipedia.org/wiki/Barn_(unit)
- atom density of silicon, https://faculty-web.msoe.edu/johnsontimoj/CE3101/files3101/silicon_crystal_structure.pdf

The silicon atom density is $5\times 10^{22} $ atoms/cm$^3$.

~~The density of  S-state electron is $10^{23} $/ cm$^{3}$~~

Electron density is around $7\times 10^{23}$ / cm$^3$

1 GeV$^{-2}$ = 0.389379 mb.
1 mb = 0.1 fm$^2$ = $10^{-27}$cm$^{2}$.

In [1]:
import numpy as np

In [2]:
distance = 0.1 # cm
silicon_density = 7E23 # electrons / cm^3


In [3]:
def xsec(energy):
    """energy is in unit of GeV
    return cross section per free electron in cm^2
    """
    alpha = 1/137.
    m = 0.511E-3 # GeV
    # xsec_in_GeVInv2 = np.pi * alpha**2/(m*energy) * np.log(2*energy/m) # GeV^-2
    gamma = energy/m
    beta = np.sqrt(1-1/gamma**2)
    xsec_in_GeVInv2 = np.pi * alpha**2/(m**2 * beta**2 * gamma * (gamma+1)) * (
            (gamma + 4 + 1/gamma)*np.log(gamma + np.sqrt(gamma**2 - 1)) - beta * (gamma+3)
            )

    GeVInv2 = 0.389379 # mb
    mb = 1E-27 # cm^2
    return xsec_in_GeVInv2 * 1 * GeVInv2 * 1 * mb
    

In [4]:
def mean_free_path(density, xsec):
    """density is in unit of electrons/cm^3
    xsec is in unit of cm^2
    return mean free path in the given density of electrons
    """
    return 1./(density * xsec)

In [5]:
def probability(mean_free_path, path):
    """mean_free_path is in unit of cm
    path is in unit of cm
    """
    return path/mean_free_path

In [6]:
energys = [4, 10, 20, 30, 50, 65, 70]# MeV
for energy in energys:
    energy_gev = energy * 1E-3 # MeV -> GeV
    e_xsec = xsec(energy_gev)
    mfp = mean_free_path(silicon_density, e_xsec)
    p = probability(mfp, distance)
    print("positron energy = {:2d} MeV.\t xsec per electron = {:4g} cm^2.\t m.f.p. = {:4g} cm.\t P under {:g} cm = {:4g} E-4".format(energy, e_xsec, mfp, distance, p*1E4))

positron energy =  4 MeV.	 xsec per electron = 8.11478e-26 cm^2.	 m.f.p. = 17.6046 cm.	 P under 0.1 cm = 56.8034 E-4
positron energy = 10 MeV.	 xsec per electron = 3.98278e-26 cm^2.	 m.f.p. = 35.8687 cm.	 P under 0.1 cm = 27.8795 E-4
positron energy = 20 MeV.	 xsec per electron = 2.32235e-26 cm^2.	 m.f.p. = 61.514 cm.	 P under 0.1 cm = 16.2565 E-4
positron energy = 30 MeV.	 xsec per electron = 1.68965e-26 cm^2.	 m.f.p. = 84.5484 cm.	 P under 0.1 cm = 11.8275 E-4
positron energy = 50 MeV.	 xsec per electron = 1.12685e-26 cm^2.	 m.f.p. = 126.775 cm.	 P under 0.1 cm = 7.88796 E-4
positron energy = 65 MeV.	 xsec per electron = 9.13133e-27 cm^2.	 m.f.p. = 156.447 cm.	 P under 0.1 cm = 6.39193 E-4
positron energy = 70 MeV.	 xsec per electron = 8.60218e-27 cm^2.	 m.f.p. = 166.071 cm.	 P under 0.1 cm = 6.02152 E-4


In [7]:
# calculate michel
# the format is (bin center, bin width, bin content)
spectrum = np.loadtxt("mutoe_michel_spectrum.txt")
ws = np.array([])
probs = np.array([])


foundEMin = False

emin = 1E20
emax = 0

psum = 0
for e, width, w in spectrum:
    if e < 2:
        continue
    if emin > e - width:
        emin = e - width
    if emax < e + width:
        emax = e + width
    energy_gev = e * 1E-3 # MeV -> GeV
    e_xsec = xsec(energy_gev)
    mfp = mean_free_path(silicon_density, e_xsec)
    p = probability(mfp, distance)
    probs = np.append(probs, p)
    ws = np.append(ws, w)
    psum = psum + p * w
    # print(e, p, w, p*w)
    # print(w)

# print(psum)
# print(probs)
# print(ws)
# print(np.sum(ws))
print("Integrating from {:.2f} to {:.2f} MeV, the probability is {:.5f}".format(emin, emax, np.sum(probs * ws)))

Integrating from 1.75 to 75.25 MeV, the probability is 0.00114


In [8]:
1-  (1-0.00114) / (1-6.02152E-4)

0.0005381720613830687